The aim of this notebook is to determine the survival rate of technologies in files after an other technology has been introduced

In [51]:
load.ivy("joda-time" % "joda-time" % "2.8")

:: problems summary ::
:::: WARNINGS
	Unable to reparse com.github.alexarchambault.jupyter#jupyter-scala-api_2.11.6;0.2.0-SNAPSHOT from sonatype-snapshots, using Fri Jun 05 10:12:58 CEST 2015

	Choosing sonatype-snapshots for com.github.alexarchambault.jupyter#jupyter-scala-api_2.11.6;0.2.0-SNAPSHOT

	Unable to reparse com.github.alexarchambault#ammonite-api_2.11.6;0.3.1-SNAPSHOT from sonatype-snapshots, using Mon Oct 12 00:17:25 CEST 2015

	Choosing sonatype-snapshots for com.github.alexarchambault#ammonite-api_2.11.6;0.3.1-SNAPSHOT

	Unable to reparse com.github.alexarchambault.jupyter#jupyter-api_2.11;0.2.0-SNAPSHOT from sonatype-snapshots, using Mon Jun 01 02:54:01 CEST 2015

	Choosing sonatype-snapshots for com.github.alexarchambault.jupyter#jupyter-api_2.11;0.2.0-SNAPSHOT



In [52]:
import java.io.File
import org.joda.time.{DateTime, DateTimeZone, Duration, Interval, Days, Weeks, Months}
import org.joda.time.format.DateTimeFormat
import scala.io.Source
import java.io.{PrintWriter,FileOutputStream}

import java.io.File
import org.joda.time.{DateTime, DateTimeZone, Duration, Interval, Days, Weeks, Months}
import org.joda.time.format.DateTimeFormat
import scala.io.Source
import java.io.{PrintWriter,FileOutputStream}

In [53]:
val directory = new File("/Users/mg/CloudStation/publications/2016/saner2016")
val first_last_file = new File(directory, "data/first-last.summary.csv")
val project_summary_file = new File(directory, "data/projects.summary.csv")
val dtf = DateTimeFormat.forPattern("yyyy-MM-dd HH:mm:ss").withZoneUTC

directory: File = /Users/mg/CloudStation/publications/2016/saner2016
first_last_file: File = /Users/mg/CloudStation/publications/2016/saner2016/data/first-last.summary.csv
project_summary_file: File = /Users/mg/CloudStation/publications/2016/saner2016/data/projects.summary.csv
dtf: org.joda.time.format.DateTimeFormatter = org.joda.time.format.DateTimeFormatter@7fab9ee2

In [83]:
val project_end = Source.fromFile(project_summary_file)
                        .getLines.drop(1)
                        .map(l => l.split(","))
                        .map(l => l(0) -> dtf.withZoneUTC.parseDateTime(l.last.substring(0,19)))
                        .toMap

project_end: Map[String, DateTime] = Map(
  "Netflix_astyanax" -> 2015-01-26T12:52:19.000Z,
  "stewartsims_KMS-dev" -> 2011-08-12T15:22:33.000Z,
  "limcheekin_vaadin-addressbook-sample" -> 2012-06-18T10:49:00.000Z,
  "apache_maven-plugins" -> 2015-03-18T22:11:07.000Z,
  "gphat_bullfinch" -> 2012-06-26T11:15:32.000Z,
  "sagioto_forum" -> 2012-07-30T18:12:20.000Z,
  "jpaoletti_java-presentation-manager" -> 2013-06-25T19:57:22.000Z,
  "qyb_sohu" -> 2012-03-30T16:17:54.000Z,
  "ebayopensource_turmeric-releng" -> 2012-01-12T09:44:28.000Z,
  "restsql_restsql" -> 2015-02-16T14:46:20.000Z,
  "openshift_kitchensink-example" -> 2014-03-31T10:02:07.000Z,
  "hamnis_httpcache4j" -> 2015-02-25T23:01:51.000Z,
  "Tagette_BossStory" -> 2014-12-22T16:07:46.000Z,
  "eclipse_cdo" -> 2015-03-18T08:05:25.000Z,
...

In [55]:
implicit def orderingByDateTime[A <: DateTime]: Ordering[A] = Ordering.by(d => d.getMillis)

case class Row(project: String, 
               file: String,
               file_end: DateTime,
               project_end: DateTime,
               technologies: Map[String, Interval])
{
    def is_entity_related(): Boolean = !technologies.isEmpty
    
    def involves(technology: String): Boolean = technologies.keySet contains technology
    
    def involves(technology: String, date: DateTime): Boolean = involves(technology) && (technologies(technology) contains date)
    
    def is_jpa(t: DateTime): Boolean = technologies.keySet contains "JPA"
    
    def is_jdbc(t: DateTime): Boolean = technologies.keySet contains "JDBC"
    
    def is_hbm(t: DateTime): Boolean = technologies.keySet contains "Hibernate"
}

object Row
{
    def apply(line: String, project_end: Map[String, DateTime]) = {
        val elements = line.split(",", -1)
        val project = elements(0)
        val file = elements(1)
        val jpa = elements(2) match {
            case "NA" => None
            case ""   => None
            case _: String => Some(new Interval( dtf.parseDateTime(elements(2)) , dtf.parseDateTime(elements(3))))  
        }
        
        val jdbc = elements(4) match {
            case "NA" => None
            case ""   => None
            case _: String => Some(new Interval(dtf.parseDateTime(elements(4)) , dtf.parseDateTime(elements(5))))  
        }
        
        val hbm = elements(6) match {
            case "NA" => None
            case ""   => None
            case _: String => Some(new Interval(dtf.parseDateTime(elements(6)) , dtf.parseDateTime(elements(7))))  
        }
        
        val technologies = scala.collection.mutable.Map[String, Interval]()
        if(jpa.isDefined) technologies.put("JPA", jpa.get)
        if(jdbc.isDefined) technologies.put("JDBC", jdbc.get)
        if(hbm.isDefined) technologies.put("Hibernate",hbm.get)
        
        if(project_end.keySet contains project) Some(new Row(project,file,project_end(project),project_end(project),technologies.toMap))
        else None
    }
}

defined function orderingByDateTime
defined class Row
defined object Row

In [56]:
val entries = Source.fromFile(first_last_file)
                    .getLines.drop(1)
                    .filterNot(_.trim.isEmpty)
                    .map(l => Row.apply(l, project_end))
                    .toList
                    .flatten

entries: List[Row] = List(
  Row(
    "codjo_codjo-sample",
    "sample-server/src/main/java/net/codjo/sample/server/broadcast/ComputedPublicationDate.java",
    2014-09-17T11:43:58.000Z,
    2014-09-17T11:43:58.000Z,
    Map("JDBC" -> 2012-07-09T14:35:56.000Z/2014-09-17T11:43:58.000Z)
  ),
  Row(
    "codjo_codjo-sample",
    "sample-server/src/main/java/net/codjo/sample/server/broadcast/BookSelector.java",
    2014-09-17T11:43:58.000Z,
    2014-09-17T11:43:58.000Z,
    Map("JDBC" -> 2012-12-18T18:18:47.000Z/2014-09-17T11:43:58.000Z)
...

Release the survival data of a sequence of values.
Each value is a duration, plus a boolean expressing if the event actually occured

In [57]:
def minDate(a: DateTime, b: DateTime): DateTime = if(a isBefore b) a else b
def maxDate(a: DateTime, b: DateTime): DateTime = if(a isAfter b) a else b 

defined function minDate
defined function maxDate

In [58]:
def release_survival(values: Seq[(Int, Boolean)], file: File) =
{
    val out = new PrintWriter(file)
    out.println("duration,event")
    values.foreach(value => {
        out.println(value._1 + "," + (if(value._2) 1 else 0))
    })
    
    out.flush()
    out.close()
}

defined function release_survival

Generate survival files

In [59]:
val technologies = Seq("JPA", "JDBC", "Hibernate")

technologies: Seq[String] = List("JPA", "JDBC", "Hibernate")

In [11]:
technologies.foreach(t1 => {
    technologies.foreach(t2 => {
        if(t1 != t2)
        {
            val candidates = entries.filter(p => (p.technologies.keySet contains t1) && 
                                                 (p.technologies.keySet contains t2) && 
                                                !(p.technologies(t1).getStart isAfter p.technologies(t2).getStart) &&
                                                 
                                                 (p.technologies(t1).getEnd isAfter p.technologies(t2).getStart)
                                           )
            
            val values = candidates.map(candidate => {
                val duration = Days.daysBetween(candidate.technologies(t2).getStart,
                                                candidate.technologies(t1).getEnd)
                                   .getDays
                assert(duration >= 0)
                val event = candidate.technologies(t1).getEnd isBefore project_end(candidate.project)
                
                (duration, event)
            })
            
            println(t1 + " -> " + t2 + " : " + candidates.size + " files" )
            release_survival(values, new File(directory, "data/survival-" + 
                                              t1.toLowerCase + "-" + t2.toLowerCase + ".csv"))
        }
    })
})

JPA -> JDBC : 664 files
JPA -> Hibernate : 185 files
JDBC -> JPA : 637 files
JDBC -> Hibernate : 155 files
Hibernate -> JPA : 112 files
Hibernate -> JDBC : 170 files


Determines the time between t2 introduction and t1 disappearance.

This time will be negative if t1 is removed before t2 is introduced.

In [12]:
technologies.foreach(t1 => {
    technologies.foreach(t2 => {
        if(t1 != t2)
        {
            val candidates = entries.filter(p => (p involves t1) && 
                                                 (p involves t2) && 
                                                !(p.technologies(t1).getStart isAfter p.technologies(t2).getStart))
            
            val values = candidates.map(candidate => {
                val duration = Days.daysBetween(candidate.technologies(t2).getStart,
                                                candidate.technologies(t1).getEnd)
                                   .getDays
                val event = candidate.technologies(t1).getEnd isBefore project_end(candidate.project)
                
                (duration, event)
            })
            
            println(t1 + " -> " + t2 + " : " + candidates.size + " files" )
            release_survival(values, new File(directory, "data/survival-" + 
                                              t1.toLowerCase + "-" + t2.toLowerCase + ".csv"))
        }
    })
})

JPA -> JDBC : 727 files
JPA -> Hibernate : 200 files
JDBC -> JPA : 703 files
JDBC -> Hibernate : 162 files
Hibernate -> JPA : 306 files
Hibernate -> JDBC : 181 files


Retrieves the first and last date a technology occured in a project

In [60]:
def limit_dates(project: String, entries: List[Row], technology: String): Option[Interval] = 
{
   val dates = entries.filter(e => e.project == project)
                      .map(e => e.technologies.get(technology))
                      .flatten
    
   if(dates.isEmpty) None
   else Some(dates.reduce((a,b) => {
              new Interval( minDate(a.getStart, b.getStart), maxDate(a.getEnd, b.getEnd))
          }))
}

defined function limit_dates

Determines the files having A when B has been introduced in the project, and finally having B
Duration: number of days between the introduction of B in the project, and the moment when B is introduced in the file
Event: occurs if B is finally introduced in the considered files

In [82]:
technologies.foreach(t2 => {
    val interesting_projects = project_end.keySet filter (p => limit_dates(p, entries, t2).isDefined)
    
   
    technologies.foreach(t1 => {
        if(t1 != t2)
        {
           val values = interesting_projects.toSeq.map(project => {
               val project_t2_start = limit_dates(project, entries, t2).get.getStart
               val interesting_files = entries.filter(entry => (entry.project == project) && 
                                                               (entry involves t1) && 
                                                               (entry.technologies(t1) contains project_t2_start)   
                                                      )
               interesting_files.map(file => {
                   if(file involves t2){
                       val duration = Days.daysBetween(project_t2_start, file.technologies(t2).getStart)
                                        .getDays 
                       (duration, true)
                   }
                   else
                   {
                       val duration = Days .daysBetween(project_t2_start, file.file_end)
                                           .getDays
                       (duration, false)
                   }
               })
           }).flatten
            
           println(t1 + " -> " + t2 + " : " + values.size + " values to register")
            
           release_survival(values, new File(directory, "data/survival-" + 
                                              t1.toLowerCase + "-after-" + t2.toLowerCase + ".csv"))   
        
           
        }
    })
})

JDBC -> JPA : 6489 values to register
Hibernate -> JPA : 653 values to register
JPA -> JDBC : 3193 values to register
Hibernate -> JDBC : 843 values to register
JPA -> Hibernate : 974 values to register
JDBC -> Hibernate : 1589 values to register


Determines the proportion of files having A when B is introduced in a project, and that finished by having B.

In [15]:
val out = new PrintWriter(new File(directory, "data/contamination-ratio.csv"))
out.println("project,t1,t2,ratio")
    
technologies.foreach(t2 => {
    val interesting_projects = project_end.keySet filter (p => limit_dates(p, entries, t2).isDefined)
    
   
    technologies.foreach(t1 => {
        if(t1 != t2)
        {
            val ratios = interesting_projects.toSeq.map(project => {
               val project_t2_start = limit_dates(project, entries, t2).get.getStart

               val interesting_files = entries.filter(entry => (entry.project == project) && 
                                                               (entry involves t1) && 
                                                               (entry.technologies(t1) contains project_t2_start))   
                
               val converted_files = interesting_files.filter(file => (file involves t2))
                
               if(interesting_files isEmpty) None
               else Some(project, (converted_files.size.toFloat / interesting_files.size.toFloat)) 
            })
            
            ratios.flatten.foreach(ratio => {
                out.println(ratio._1 + "," + t1 + "," + t2 + "," + ratio._2)
            })
            
        }
    })
})

out.close()

out: PrintWriter = java.io.PrintWriter@6bdab480

# Contamination evolution

Determines the proportion of files having A when B has been introduced in a project, and that finish by having B before a given duration

In [121]:
val out = new PrintWriter(new File(directory, "data/contamination-ratio-evolution.csv"))
out.println("project,t1,t2,duration,ratio,cumratio")
val MAX_WEEK = 15

technologies.foreach(t2 => {
    val interesting_projects = project_end.keySet.filter (p => limit_dates(p, entries, t2).isDefined)

    technologies.foreach(t1 => {
        if(t2 != t1)
        {
            val weeks = interesting_projects.toSeq.map(project => {
                val project_t2_start = limit_dates(project, entries, t2).get.getStart
                val interesting_files = entries.filter(entry =>(entry.project == project) && 
                                                               (entry involves t1) && 
                                                               (entry.technologies(t1) contains project_t2_start))   
           
                val durations = interesting_files.map(file => {
                    if(!file.involves(t2)) None
                    else Some(Weeks.weeksBetween(project_t2_start, file.technologies(t2).getStart).getWeeks)
                }) 
                
                project -> (interesting_files.size, durations.flatten)
            }).toMap 
                       
            weeks.filterNot(w => w._2._1 == 0).foreach(w => {
                val project = w._1
                val nb_files = w._2._1.toFloat
                val durations = w._2._2
                
                (0 to MAX_WEEK).foreach(week => {
                    val ratio = durations.count(d => d == week).toFloat / nb_files.toFloat
                    val cumRatio = durations.count(d => d <= week).toFloat / nb_files.toFloat
                    out.println(project + "," + t1 + "," + t2 + "," + week + "," + ratio + "," + cumRatio)
                })
            })
        }
    })
})

out.close

out: PrintWriter = java.io.PrintWriter@73411c98
MAX_WEEK: Int = 15

# Ratio evolution

Determines the proportion of files having A when B has been introduced in a project, and that 
 - don't have neither A nor B (suppression)
 - have A and B (completion)
 - don't have A and have B (replacement)
 - have A and don't have B (residual)
 
These proportions are evaluated over time, for predefined weeks following the introduction of B in the project.
 
Only projects in which A has been introduced at least 1 week before B are used.
If a project finished before one of the considered time, there is no output for this project and this time.

In [126]:
case class Project(name: String, files: Seq[Row])
{
    def start = files.map(f => )
}

val projects = entries.groupBy(_.project).map(e => Project(e._1, e._2))

defined class Project
projects: collection.immutable.Iterable[$user.Project] = List(
  Project(
    "Netflix_astyanax",
    List(
      Row(
        "Netflix_astyanax",
        "src/test/java/com/netflix/astyanax/entitystore/DoubleIdColumnEntity.java",
        2015-01-26T12:52:19.000Z,
        2015-01-26T12:52:19.000Z,
        Map("JPA" -> 2013-01-28T10:07:04.000Z/2013-02-11T09:47:00.000Z)
      ),
      Row(
        "Netflix_astyanax",
        "src/test/java/com/netflix/astyanax/entitystore/TtlEntity.java",
        2015-01-26T12:52:19.000Z,
...

In [125]:
val out = new PrintWriter(new File(directory, "data/partition-ratio-evolution.csv"))
out.println("project,interestingfiles,t1,t2,duration,suppression,completion,replacement,residual")
val interesting_weeks = Seq(0,1,8,15)
val technologies = Seq("JPA", "Hibernate")

technologies.foreach(t2 => {
    val interesting_projects = project_end.keySet.filter (p => limit_dates(p, entries, t2).isDefined)

    technologies.foreach(t1 => {
        if(t1 != t2)
        {   
            
            val selected_projects = interesting_projects.filter(project => {
                val res_t2 = limit_dates(project, entries, t2).get
                val project_t2_start = res_t2.getStart
                val project_t2_end = res_t2.getEnd
                
                val res_t1 = limit_dates(project, entries, t1)
                val project_t1_start = res_t1.map(_.getStart)
                val project_t1_end = res_t1.map(_.getEnd)
                   
                // Filter projects in which A has been introduced at least 1 week before B,
                // and B has been introduced while A was still present
                project_t1_start.isDefined && 
                (project_t1_start.get isBefore (project_t2_start minus Weeks.ONE )) && 
                (project_t2_start isBefore project_t1_end.get)
            })
            
            println(t1 + " -> " + t2 + " : " + selected_projects.size + " selected projects")
            
            selected_projects.toSeq.foreach(project => {
                val project_t2_start = limit_dates(project, entries, t2).get.getStart
                val interesting_files = entries.filter(file => (file.project == project) && 
                                                               (file involves t1) && 
                                                               (file.technologies(t1) contains project_t2_start))
                
                val file_size = interesting_files.size.toFloat
                
                val global_end = project_end(project)
                
                interesting_weeks.foreach(week => {
                    val date = project_t2_start plus Weeks.weeks(week)
                    
                    if(date isBefore global_end) // The project is not finished
                    {
                        val suppression = interesting_files.count(file => (!file.involves(t1, date)) && (!file.involves(t2, date)))
                        val suppression_ratio = suppression.toFloat / file_size

                        val completion  = interesting_files.count(file => file.involves(t1, date) && file.involves(t2, date))
                        val completion_ratio = completion.toFloat / file_size

                        val replacement = interesting_files.count(file => (!file.involves(t1, date)) && file.involves(t2, date))
                        val replacement_ratio = replacement.toFloat / file_size

                        val residual    = file_size - suppression - completion - replacement
                        val residual_ratio = residual.toFloat / file_size                                          

                        assert(residual >= 0.0)

                        out.println(project + "," + interesting_files.size + "," + 
                                    t1 + "," + t2 + "," + week + "," +
                                    suppression_ratio + "," + completion_ratio + "," + 
                                    replacement_ratio + "," + residual_ratio)  
                    }
 
                })

            })
        }
    })
})
        
out.close()


false
true
false
false
true
true
true
false
true
true
false
true
true
false
false
true
false
true
true
true
false
false
false
false
true
false
false
false
false
true
false
false
false
false
false
true
true
true
false
false
false
true
true
true
false
true
true
false
false
true
false
false
false
true
false
true
true
true
false
true
false
false
false
true
true
true
false
true
false
false
false
false
Hibernate -> JPA : 21 selected projects
false
false
false
false
false
false
false
true
false
false
false
false
false
false
true
false
false
false
false
false
false
false
true
false
false
false
false
true
false
false
true
true
true
true
true
false
false
false
false
true
true
false
false
false
true
false
false
true
false
false
false
false
true
false
false
false
false
false
false
false
true
false
false
false
false
false
false
false
false
true
false
true
JPA -> Hibernate : 16 selected projects


out: PrintWriter = java.io.PrintWriter@1b8aee76
interesting_weeks: Seq[Int] = List(0, 1, 8, 15)
technologies: Seq[String] = List("JPA", "Hibernate")

# Number of files and projects having each of the technologies, over time

In [37]:
val first_date = entries.flatMap(e => e.technologies.map(t => t._2.getStart)).minBy(date => date.getMillis)
val last_date = entries.flatMap(e => e.technologies.map(t => t._2.getEnd)).maxBy(date => date.getMillis)

val out = new PrintWriter(new File(directory, "data/techno-usage-evolution.csv"))
var current = first_date
val dtf_short = DateTimeFormat.forPattern("yyyy-MM");
out.println("date,jpa-files,jdbc-files,hibernate-files,jpa-projects,jdbc-projects,hibernate-projects")
while(current isBefore last_date)
{
    val results_files = technologies.map(techno => {
        techno -> entries.count(entry => (entry involves techno) && (entry.technologies(techno) contains current))
    }).toMap
    
    val results_projects = technologies.map(techno => {
        techno -> entries.filter(entry => (entry involves techno) && (entry.technologies(techno) contains current))
                         .map(_.project).distinct.size   
    }).toMap
    
    out.println(dtf_short.print(current) + "," + results_files("JPA")    + "," + results_files("JDBC")    + "," + results_files("Hibernate") + "," +
                                       results_projects("JPA") + "," + results_projects("JDBC") + "," + results_projects("Hibernate"))
    current = current plus Months.ONE
}

out.close()

out: PrintWriter = java.io.PrintWriter@1c734f10
current: DateTime = 2015-04-01T00:00:00.000Z
dtf: org.joda.time.format.DateTimeFormatter = org.joda.time.format.DateTimeFormatter@613c025d